In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Playground

In [2]:
# Example (playground)
# Bag of Words example

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet,'
                 'and one and one is two'])
bag = count.fit_transform(docs)

# Print stored vocabularies into dictionaries (ascending order 
#   for mapping purpose)
print(count.vocabulary_)

# print the vectorization result
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [3]:
# Example (playground)
# TFIDF Example

from sklearn.feature_extraction.text import TfidfTransformer

count = CountVectorizer()
# Transform the raw text data into a document-term matrix (DTM)
dtm = count.fit_transform(docs)

# Create a TfidfTransformer instance with desired parameters
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

# Fit and transform the DTM using the TfidfTransformer
tfidf_matrix = tfidf.fit_transform(dtm)

# Convert the sparse matrix to a dense array and print
dense_array = tfidf_matrix.toarray()
np.set_printoptions(precision=2)


# Print stored vocabularies into dictionaries (ascending order 
#   for mapping purpose)
print(count.vocabulary_)

# print the vectorization result
print(dense_array)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [4]:
# Example (playground)
# Tokenizer Example

# 1. Tokenizer using simple spliter
def tokenizer(text):
    return text.split()

text = 'runners like running and thus they run'
print(tokenizer(text))

# 2. Tokenizer using Porter Stemmer algorithm
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter(text))

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']
['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


In [5]:
# Example (playground)
# stop-word removal Example

# We can use NLTK library that contains stop-words (in this case in English)

import nltk
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
result = [w for w in tokenizer_porter(text) if w not in stop_words]
print(result)

['runner', 'like', 'run', 'thu', 'run']


# Training a logistic regression model for document classification

In [6]:
# basepath = 'data/aclImdb'

# labels = {'pos': 1, 'neg': 0}
# data = {'review': [], 'sentiment': []}

# for s in ('test', 'train'):
#     for l in labels.keys():
#         path = os.path.join(basepath, s, l)
#         for file in sorted(os.listdir(path)):
#             with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
#                 txt = infile.read()
#                 data['review'].append(txt)
#                 data['sentiment'].append(labels[l])

# df = pd.DataFrame(data)

In [7]:
# np.random.seed(0)
# df = df.reindex(np.random.permutation(df.index))
# df.to_csv('data/movie_data.csv', index=False, encoding='utf-8')

In [8]:
df = pd.read_csv('data/movie_data.csv', encoding='utf-8')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [9]:
# Cleaning text data

# 1. Shows HTML format
print(df.loc[0, 'review'][-50:])

is seven.<br /><br />Title (Brazil): Not Available


In [10]:
# Data cleansing

def preprocessor(text):
    # 1. Removes any HTML tags from the text 
    text = re.sub('<[^>]*>', '', text)
    # 2. Finds emoticons in the text using
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # 3. Converts the text to lowercase and removes any non-word characters 
    #    (characters other than letters, digits, and underscores) 
    #    by replacing them with a space.
    # 4. Joins the found emoticons into a single string with spaces between them, 
    #    and removes any '-' characters from the emoticons.
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

# Example
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [11]:
df['review'] = df['review'].apply(preprocessor)

In [12]:
X_train = df.loc[:100, 'review'].values
y_train = df.loc[:100, 'sentiment'].values
X_test = df.loc[100:, 'review'].values
y_test = df.loc[100:, 'sentiment'].values

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
              'vect__stop_words': [stop_words, None],
               'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
              'vect__stop_words': [stop_words, None],
               'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
             ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0,
                                               solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\

[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

...

[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x0000014BD385C180>, vect__use_idf=False; total time=   0.6s


C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x0000014BD385C180>, vect__use_idf=False; total time=   0.5s


C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've"...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x0000014BD385C360>,
                                              <function tokenizer_porter at 0x0000014BD385C180>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [14]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x0000014BD385C180>} 


In [15]:
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy: 0.692


In [16]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.603


# Working with bigger data – online algorithms and out-of-core learning

1. Define preprocessing and tokenizer function.
2. Generate stream function that reads in and returns one document at a time.
3. Define a function that will take a document stream and return a particular number of documents specified by its size (returns a chunks).
4. Train model in each chunks.

NOTE: Recall concept Stochastic Gradient Descent.

In [17]:
from nltk.corpus import stopwords
import csv

stop_words = stopwords.words('english')

def tokenizer(text):
    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)
    # Find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    # Convert to lowercase and remove non-word characters
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    # Tokenize by splitting the text into words
    tokenized = text.split()
    return tokenized
        
def stream_docs(path):
    with open(path, 'r', newline='', encoding='utf-8') as file:
        next(file) # skip header
        # Create a CSV reader object
        csv_reader = csv.reader(file)
        # Read the contents of the file into a list of rows
        csv_data = list(csv_reader)
        for line in csv_data:
            text, label = line[0], int(line[-1])
            
            yield text, label

# Test stream_docs function
print(next(stream_docs(path='data/movie_data.csv')))

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    
    return docs, y

('In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich fami

In [18]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log_loss', random_state=1)
doc_stream = stream_docs(path='data/movie_data.csv')

In [19]:
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

In [20]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test)}')

Accuracy: 0.83


In [21]:
# Melakukan partial fit
clf = clf.partial_fit(X_test, y_test)

NOTE: 
- CountVectorizer tidak bisa digunakan pada out-of-core learning. Perlu seluruh data, tidak bisa parsial.
- Vectorizer dapat menggunakan HashingVectorizer.
- Dengan memilih nilai "n_features" pada HashingVectorizer, kita mereduksi peluang terjadinya "hash collisions" namun kita meningkatkan koefisien pada model kita (kasus ini Logistic Regression).

# Topic Modeling with Latent Dirichlet Allocation (LDA)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

# Define CountVectorizer with specified parameters
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)

# Fit and transform the 'review' column of DataFrame
X = count.fit_transform(df['review'].values)


NOTE:
- We set the maximum document frequency of words to be considered to 10 percent (max_df = .1)
- We limited the number of words to be considered to most frequently occuring 5000 words (max_features=5000)

In [23]:
from sklearn.decomposition import LatentDirichletAllocation

# Define LatentDirichletAllocation with specified parameters
lda = LatentDirichletAllocation(n_components=10, random_state=123, 
                                learning_method='batch')

# Fit and transform the data using LDA
X_topics = lda.fit_transform(X)


In [24]:
lda.components_.shape

(10, 5000)

In [25]:
n_top_words = 5
feature_names = count.get_feature_names_out()

# Iterate over each topic and print the top words
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))


Topic 1:
horror original comedy black house
Topic 2:
worst minutes guy script money
Topic 3:
book dvd read version watched
Topic 4:
family performance father beautiful mother
Topic 5:
series episode tv kids comedy
Topic 6:
murder police wife john plays
Topic 7:
documentary camera effects audience sense
Topic 8:
music song songs musical role
Topic 9:
horror effects guy dead budget
Topic 10:
action war game fight american


In [26]:
horror = X_topics[:, 5].argsort()[::-1]

# Print the top 3 horror movie reviews
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')

# NOTE:
# - horror movies belong to category 6 at index position 5


Horror movie #1:
 spoilers extremely brutal police drama set in san francisco involving a sting operation that goes terribly wrong a cop det falon sam elliott mistakenly and savagely beats to death an undercover policeman winch mike watson thinking that he murdered his partner det sam levinson mike burstyn a partner ...

Horror movie #2:
this first rate western tale of the gold rush brings great excitement romance and james stewart to the screen the far country is the only one out of all five stewart mann westerns that is often overlooked stewart yet again puts a new look on the ever present personalities he had in the five stewart  ...

Horror movie #3:
the fourth of five westerns anthony mann did with james stewart this one involves a hard bitten cattleman named jeff webster who takes a cattle drive from wyoming to alaska via seattle he hooks up in seattle with his partners ben tatum walter brennan and rube morris jay c flippen that he has sent ah ...


In [ ]:
horror